# 🏔️ MODIS MOD10A1 Snow Albedo Analysis - Saskatchewan Glacier
## Interactive Python/Geemap Implementation

---

### 📊 **Research-Grade Snow Albedo Analysis (2010-2024)**

**Purpose**: Extract high-quality snow albedo data with comprehensive QA filtering using Python, geemap, and advanced statistical analysis.

### 🎯 **Key Features**:
- 📈 **Deep Statistical Analysis**: Sen's slope, Mann-Kendall trends, change points, anomaly detection
- ⚡ **Interactive Widgets**: Real-time parameter optimization with ipywidgets
- ☁️ **Comprehensive QA**: MOD10A1 v6.1 cloud detection and quality filtering
- 🔬 **Research-Grade Filtering**: NDSI snow + glacier fraction + quality masks
- 📊 **Enhanced Exports**: Annual summaries, daily time series, CSV generation
- 🐍 **Python Ecosystem**: Integration with scipy, pandas, matplotlib, plotly

### 🔍 **Quality Control Methods**:
- 🌙 **Basic QA**: Excludes night, ocean, poor quality pixels
- 🚩 **Algorithm Flags**: Comprehensive 8-bit QA filtering
- 🗺️ **Spatial Filter**: Glacier fraction thresholds
- ⏰ **Temporal Filter**: Statistical reliability minimums

### 📅 **Analysis Period**: June-September (Extended melt season)
### 🏔️ **Default Settings**: NDSI ≥0, Glacier ≥75%, Good+ QA quality

---

## 📦 **Phase 1: Environment Setup & Imports**

In [ ]:
# Core geospatial and Earth Engine libraries
import ee
import geemap
import geemap.colormaps as cm

# Interactive widgets and visualization
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display, HTML, clear_output

# Scientific computing and statistics
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy import signal
import pymannkendall as mk

# Visualization libraries
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import seaborn as sns

# Utilities
import datetime
from datetime import datetime, timedelta
import warnings
import json
import os
from typing import Dict, List, Tuple, Optional

# Configure matplotlib and seaborn
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

print("📦 Libraries imported successfully!")
print(f"🌍 Earth Engine version: {ee.__version__}")
print(f"🗺️ Geemap version: {geemap.__version__}")

In [ ]:
# Authenticate Google Earth Engine (run this first if needed)
# Uncomment and run the line below if you need to authenticate:
# ee.Authenticate()

# Note: After authentication, restart the kernel and re-run all cells

In [ ]:
# Authenticate and Initialize Earth Engine
try:
    # Try to initialize first (if already authenticated)
    ee.Initialize()
    print("✅ Earth Engine initialized successfully!")
except Exception as e:
    print("🔑 Earth Engine authentication required...")
    print("Please run the following in a separate code cell:")
    print("ee.Authenticate()")
    print("Then re-run this cell.")
    print(f"Error: {e}")

## ⚙️ **Configuration Parameters**

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════════════
# CONFIGURATION PARAMETERS - Research-grade conservative settings
# ═══════════════════════════════════════════════════════════════════════════════════════

# Study period and temporal settings
STUDY_YEARS = list(range(2010, 2025))  # 2010-2024
SUMMER_START_MONTH = 6  # June (extended melt season)
SUMMER_END_MONTH = 9    # September
USE_PEAK_MELT_ONLY = False  # False = June-Sept, True = July-Sept

# Filtering thresholds (conservative defaults)
NDSI_SNOW_THRESHOLD = 0      # Minimum NDSI Snow Cover (index 0-100)
GLACIER_FRACTION_THRESHOLD = 75  # Minimum glacier fraction (%) 
MIN_PIXEL_THRESHOLD = 10     # Minimum pixels for statistical reliability

# Glacier fraction classification thresholds
FRACTION_THRESHOLDS = [0.25, 0.50, 0.75, 0.90]

# Class names for different glacier fraction categories
FRACTION_CLASS_NAMES = [
    'glacier_0_25pct', 'glacier_25_50pct', 'glacier_50_75pct', 
    'glacier_75_90pct', 'glacier_90_100pct'
]

ANNUAL_CLASS_NAMES = [
    'glacier_0_25pct_high_snow', 'glacier_25_50pct_high_snow', 
    'glacier_50_75pct_high_snow', 'glacier_75_90pct_high_snow', 
    'glacier_90_100pct_high_snow'
]

# Quality Assessment configuration (conservative approach)
QA_CONFIG = {
    'basic_level': 'good',              # Good quality+ (0-1)
    'exclude_inland_water': True,       # Exclude water/lakes
    'exclude_visible_screen_fail': True,    # CRITICAL - corrupted data
    'exclude_ndsi_screen_fail': True,       # CRITICAL - unreliable NDSI
    'exclude_temp_height_fail': True,       # IMPORTANT - atypical conditions
    'exclude_swir_anomaly': True,           # IMPORTANT - optical anomalies
    'exclude_probably_cloudy': True,        # CRITICAL - v6.1 cloud detection
    'exclude_probably_clear': False,        # Keep clear sky pixels
    'exclude_high_solar_zenith': True       # IMPORTANT - poor lighting
}

# Asset paths
GLACIER_ASSET_PATH = 'projects/tofunori/assets/Saskatchewan_glacier_2024_updated'

print("⚙️ Configuration loaded:")
print(f"   📅 Study period: {STUDY_YEARS[0]}-{STUDY_YEARS[-1]}")
print(f"   🌞 Season: {'July-September' if USE_PEAK_MELT_ONLY else 'June-September'}")
print(f"   ❄️ NDSI threshold: ≥{NDSI_SNOW_THRESHOLD}")
print(f"   🏔️ Glacier fraction: ≥{GLACIER_FRACTION_THRESHOLD}%")
print(f"   📊 Min pixels: ≥{MIN_PIXEL_THRESHOLD}")
print(f"   ✅ QA level: {QA_CONFIG['basic_level']}+ quality")

## 🏔️ **Phase 2: Data Loading & Glacier Setup**

In [ ]:
# Load Saskatchewan glacier asset
print("🏔️ Loading Saskatchewan glacier asset...")

try:
    saskatchewan_glacier = ee.Image(GLACIER_ASSET_PATH)
    glacier_mask = saskatchewan_glacier.gt(0)
    
    # Create glacier geometry for analysis - Fixed reduceToVectors call
    glacier_geometry = glacier_mask.reduceToVectors(
        reducer=ee.Reducer.count(),  # Added explicit reducer parameter
        scale=30,
        maxPixels=1e6,
        tileScale=2
    ).geometry()
    
    print("✅ Glacier asset loaded successfully!")
    
    # Get glacier area information
    glacier_area = glacier_mask.multiply(ee.Image.pixelArea()).reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=glacier_geometry,
        scale=30,
        maxPixels=1e9
    )
    
    # Convert to client-side for display
    area_info = glacier_area.getInfo()
    glacier_area_km2 = area_info['constant'] / 1e6
    
    print(f"📏 Glacier area: {glacier_area_km2:.2f} km²")
    
except Exception as e:
    print(f"❌ Error loading glacier asset: {e}")
    print("Please check the asset path and permissions.")

In [ ]:
# Compute static glacier fraction (optimization for repeated use)
print("🔄 Computing static glacier fraction...")

try:
    # Get MODIS projection reference
    modis_reference = ee.ImageCollection('MODIS/061/MOD10A1') \
        .filterDate('2020-01-01', '2020-01-02') \
        .first()
    
    modis_projection = modis_reference.projection()
    
    # Create 30m raster and reproject to MODIS 500m
    raster30 = ee.Image.constant(1) \
        .updateMask(glacier_mask) \
        .unmask(0) \
        .reproject(modis_projection, None, 30)
    
    # Compute glacier fraction at 500m resolution
    STATIC_GLACIER_FRACTION = raster30.reduceResolution({
        'reducer': ee.Reducer.mean(),
        'maxPixels': 1024
    }).reproject(modis_projection, None, 500)
    
    # Get fraction statistics
    fraction_stats = STATIC_GLACIER_FRACTION.reduceRegion({
        'reducer': ee.Reducer.minMax(),
        'geometry': glacier_geometry,
        'scale': 500,
        'maxPixels': 1e9,
        'tileScale': 2
    })
    
    stats_info = fraction_stats.getInfo()
    print(f"✅ Glacier fraction computed successfully!")
    print(f"📊 Fraction range: {stats_info['constant_min']:.3f} - {stats_info['constant_max']:.3f}")
    
except Exception as e:
    print(f"❌ Error computing glacier fraction: {e}")

## 🛠️ **Phase 3: Quality Assessment Functions**

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════════════
# QUALITY ASSESSMENT FUNCTIONS
# Comprehensive QA filtering based on MOD10A1 v6.1 documentation
# ═══════════════════════════════════════════════════════════════════════════════════════

def get_basic_qa_mask(image: ee.Image, level: str = 'good') -> ee.Image:
    """
    Create Basic QA mask based on quality level.
    
    Args:
        image: MOD10A1 image with NDSI_Snow_Cover_Basic_QA band
        level: Quality level ('best', 'good', 'ok', 'all')
    
    Returns:
        Binary mask (1=keep, 0=exclude)
    """
    basic_qa = image.select('NDSI_Snow_Cover_Basic_QA')
    
    # Quality thresholds based on MOD10A1 documentation
    quality_thresholds = {
        'best': 0,  # Best quality only
        'good': 1,  # Good quality and above (0-1)
        'ok': 2,    # OK quality and above (0-2)
        'all': 3    # All quality levels (0-3)
    }
    
    threshold = quality_thresholds.get(level, 1)
    quality_mask = basic_qa.lte(threshold)
    
    # Always exclude night (211) and ocean (239)
    exclude_mask = basic_qa.neq(211).And(basic_qa.neq(239))
    
    return quality_mask.And(exclude_mask)


def get_algorithm_flags_mask(image: ee.Image, config: Dict) -> ee.Image:
    """
    Create Algorithm Flags QA mask based on configuration.
    
    Args:
        image: MOD10A1 image with NDSI_Snow_Cover_Algorithm_Flags_QA band
        config: Dictionary with boolean flags for each bit
    
    Returns:
        Binary mask (1=keep, 0=exclude)
    """
    alg_flags = image.select('NDSI_Snow_Cover_Algorithm_Flags_QA')
    mask = ee.Image(1)
    
    # QA bit mapping for MOD10A1 v6.1 Algorithm Flags
    qa_bits = {
        'exclude_inland_water': 0,        # Bit 0: Inland water
        'exclude_visible_screen_fail': 1,  # Bit 1: Low visible screen failure
        'exclude_ndsi_screen_fail': 2,     # Bit 2: Low NDSI screen failure
        'exclude_temp_height_fail': 3,     # Bit 3: Temperature/height screen failure
        'exclude_swir_anomaly': 4,         # Bit 4: SWIR reflectance anomaly
        'exclude_probably_cloudy': 5,      # Bit 5: Probably cloudy (v6.1)
        'exclude_probably_clear': 6,       # Bit 6: Probably clear (v6.1)
        'exclude_high_solar_zenith': 7     # Bit 7: High solar zenith (>70°)
    }
    
    # Apply each flag based on configuration
    for flag_name, bit_position in qa_bits.items():
        if config.get(flag_name, False):
            bit_mask = ee.Number(2).pow(bit_position).int()
            mask = mask.And(alg_flags.bitwiseAnd(bit_mask).eq(0))
    
    return mask


def create_comprehensive_quality_mask(image: ee.Image, config: Dict) -> ee.Image:
    """
    Create comprehensive quality mask combining Basic QA and Algorithm Flags.
    
    Args:
        image: MOD10A1 image
        config: QA configuration dictionary
    
    Returns:
        Combined quality mask
    """
    basic_mask = get_basic_qa_mask(image, config.get('basic_level', 'good'))
    flags_mask = get_algorithm_flags_mask(image, config)
    
    return basic_mask.And(flags_mask)


def create_fraction_masks(fraction_image: ee.Image, thresholds: List[float]) -> Dict[str, ee.Image]:
    """
    Create glacier fraction classification masks.
    
    Args:
        fraction_image: Glacier fraction image (0-1)
        thresholds: List of fraction thresholds [0.25, 0.50, 0.75, 0.90]
    
    Returns:
        Dictionary of fraction masks
    """
    masks = {}
    masks['glacier_0_25pct'] = fraction_image.gt(0).And(fraction_image.lt(thresholds[0]))
    masks['glacier_25_50pct'] = fraction_image.gte(thresholds[0]).And(fraction_image.lt(thresholds[1]))
    masks['glacier_50_75pct'] = fraction_image.gte(thresholds[1]).And(fraction_image.lt(thresholds[2]))
    masks['glacier_75_90pct'] = fraction_image.gte(thresholds[2]).And(fraction_image.lt(thresholds[3]))
    masks['glacier_90_100pct'] = fraction_image.gte(thresholds[3])
    
    return masks


print("🛠️ Quality assessment functions defined successfully!")
print("   ✅ Basic QA filtering")
print("   ✅ Algorithm Flags filtering (8-bit)")
print("   ✅ Comprehensive QA masking")
print("   ✅ Glacier fraction classification")

## 📊 **Phase 4: Data Processing Functions**

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════════════
# DATA PROCESSING FUNCTIONS
# Core functions for annual and daily albedo analysis
# ═══════════════════════════════════════════════════════════════════════════════════════

def calculate_annual_albedo_statistics(year: int, 
                                     ndsi_threshold: int = NDSI_SNOW_THRESHOLD,
                                     glacier_threshold: int = GLACIER_FRACTION_THRESHOLD,
                                     min_pixels: int = MIN_PIXEL_THRESHOLD,
                                     qa_config: Dict = QA_CONFIG) -> ee.Feature:
    """
    Calculate annual albedo statistics for high snow cover areas.
    
    Args:
        year: Analysis year
        ndsi_threshold: Minimum NDSI snow cover threshold
        glacier_threshold: Minimum glacier fraction percentage
        min_pixels: Minimum pixels for statistical reliability
        qa_config: Quality assessment configuration
    
    Returns:
        Feature with annual statistics
    """
    # Define date range
    start_month = 7 if USE_PEAK_MELT_ONLY else SUMMER_START_MONTH
    year_start = ee.Date.fromYMD(year, start_month, 1)
    year_end = ee.Date.fromYMD(year, SUMMER_END_MONTH, 30)
    
    # Load and filter MOD10A1 collection
    collection = ee.ImageCollection('MODIS/061/MOD10A1') \
        .filterDate(year_start, year_end) \
        .filterBounds(glacier_geometry) \
        .select(['NDSI_Snow_Cover', 'Snow_Albedo_Daily_Tile', 
                'NDSI_Snow_Cover_Basic_QA', 'NDSI_Snow_Cover_Algorithm_Flags_QA']) \
        .map(lambda img: img.clip(glacier_geometry))
    
    # Process each image
    def process_image(img):
        snow_cover = img.select('NDSI_Snow_Cover')
        snow_albedo = img.select('Snow_Albedo_Daily_Tile')
        
        # Create quality masks
        qa_mask = create_comprehensive_quality_mask(img, qa_config)
        ndsi_mask = snow_cover.gte(ndsi_threshold)
        glacier_mask = STATIC_GLACIER_FRACTION.gte(glacier_threshold / 100)
        albedo_mask = snow_albedo.lte(100)
        
        # Combined mask
        combined_mask = qa_mask.And(ndsi_mask).And(glacier_mask).And(albedo_mask)
        
        # Scale albedo to 0-1 range
        albedo_scaled = snow_albedo.divide(100).updateMask(combined_mask).rename('albedo')
        
        # Create fraction masks and apply to albedo
        fraction_masks = create_fraction_masks(STATIC_GLACIER_FRACTION, FRACTION_THRESHOLDS)
        
        masked_albedos = []
        for i, class_name in enumerate(ANNUAL_CLASS_NAMES):
            fraction_key = FRACTION_CLASS_NAMES[i]
            masked_albedo = albedo_scaled.updateMask(fraction_masks[fraction_key]).rename(class_name)
            masked_albedos.append(masked_albedo)
        
        # Add pixel count band
        pixel_count = combined_mask.rename('high_snow_pixel_count')
        
        return ee.Image.cat(masked_albedos + [pixel_count])
    
    processed_collection = collection.map(process_image)
    
    # Calculate statistics
    albedo_means = processed_collection.select(ANNUAL_CLASS_NAMES).mean()
    pixel_count_total = processed_collection.select('high_snow_pixel_count').sum()
    
    # Reduce to get statistics
    all_stats = albedo_means.reduceRegion({
        'reducer': ee.Reducer.mean().combine(ee.Reducer.stdDev(), '', True).combine(ee.Reducer.count(), '', True),
        'geometry': glacier_geometry,
        'scale': 500,
        'maxPixels': 1e9,
        'tileScale': 4
    })
    
    filtered_pixel_stats = pixel_count_total.reduceRegion({
        'reducer': ee.Reducer.sum(),
        'geometry': glacier_geometry,
        'scale': 500,
        'maxPixels': 1e9,
        'tileScale': 4
    })
    
    # Build properties
    total_pixels = filtered_pixel_stats.get('high_snow_pixel_count')
    sufficient_pixels = ee.Number(total_pixels).gte(min_pixels)
    
    properties = {
        'year': year,
        'ndsi_snow_threshold': ndsi_threshold,
        'glacier_fraction_threshold': glacier_threshold,
        'min_pixel_threshold': min_pixels,
        'peak_melt_only': USE_PEAK_MELT_ONLY,
        'total_filtered_pixels': total_pixels,
        'sufficient_pixels': sufficient_pixels
    }
    
    # Add class-specific statistics
    for class_name in ANNUAL_CLASS_NAMES:
        class_count = all_stats.get(class_name + '_count')
        class_sufficient = ee.Number(class_count).gte(min_pixels)
        
        properties[class_name + '_mean'] = ee.Algorithms.If(class_sufficient, 
                                                          all_stats.get(class_name + '_mean'), None)
        properties[class_name + '_stdDev'] = ee.Algorithms.If(class_sufficient, 
                                                            all_stats.get(class_name + '_stdDev'), None)
        properties[class_name + '_count'] = class_count
        properties[class_name + '_sufficient_pixels'] = class_sufficient
    
    return ee.Feature(None, properties)


print("📊 Data processing functions defined successfully!")
print("   ✅ Annual albedo statistics calculation")
print("   ✅ Quality mask integration")
print("   ✅ Glacier fraction classification")
print("   ✅ Statistical reliability validation")

## 🗺️ **Phase 5: Interactive Map Setup**

In [ ]:
# Create interactive map with geemap
print("🗺️ Setting up interactive map...")

# Initialize map centered on Saskatchewan Glacier
Map = geemap.Map(
    center=[52.15, -117.28],  # Saskatchewan Glacier coordinates
    zoom=12,
    height='600px',
    width='100%'
)

# Set satellite basemap for better glacier visualization
Map.add_basemap('SATELLITE')

# Add glacier mask to map
glacier_vis = {
    'palette': ['orange'],
    'opacity': 0.7
}

Map.addLayer(glacier_mask.selfMask(), glacier_vis, 'Saskatchewan Glacier Mask')

# Add glacier fraction layer (initially hidden)
fraction_vis = {
    'min': 0,
    'max': 1,
    'palette': ['red', 'orange', 'yellow', 'green', 'cyan', 'blue']
}

Map.addLayer(STATIC_GLACIER_FRACTION, fraction_vis, 'Glacier Fraction', False)

print("✅ Interactive map setup complete!")
print("   🗺️ Centered on Saskatchewan Glacier")
print("   🛰️ Satellite basemap enabled")
print("   🏔️ Glacier mask and fraction layers added")

# Display the map
Map

## 🎛️ **Phase 6: Interactive Widgets & Controls**

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════════════
# INTERACTIVE WIDGETS - Replace GEE UI with ipywidgets
# ═══════════════════════════════════════════════════════════════════════════════════════

# Global variables for current analysis
current_image = None
current_stats = {}
current_qa_retention = {}

# Create date picker widget
date_picker = widgets.DatePicker(
    description='Analysis Date:',
    value=datetime(2023, 8, 7),
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

# Parameter control sliders
ndsi_slider = widgets.IntSlider(
    value=NDSI_SNOW_THRESHOLD,
    min=0,
    max=100,
    step=5,
    description='NDSI Threshold:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

glacier_fraction_slider = widgets.IntSlider(
    value=GLACIER_FRACTION_THRESHOLD,
    min=0,
    max=100,
    step=5,
    description='Glacier Fraction (%):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

min_pixel_slider = widgets.IntSlider(
    value=MIN_PIXEL_THRESHOLD,
    min=0,
    max=100,
    step=1,
    description='Min Pixels:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

# QA Level dropdown
qa_level_dropdown = widgets.Dropdown(
    options=[
        ('Best quality only (0)', 'best'),
        ('Good quality+ (0-1)', 'good'),
        ('OK quality+ (0-2)', 'ok'),
        ('All quality levels (0-3)', 'all')
    ],
    value='good',
    description='Basic QA Level:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

# Algorithm flags checkboxes
flag_checkboxes = {
    'exclude_inland_water': widgets.Checkbox(
        value=QA_CONFIG['exclude_inland_water'],
        description='Bit 0: Exclude Inland Water',
        style={'description_width': 'initial'}
    ),
    'exclude_visible_screen_fail': widgets.Checkbox(
        value=QA_CONFIG['exclude_visible_screen_fail'],
        description='Bit 1: Exclude Visible Screen Fail',
        style={'description_width': 'initial'}
    ),
    'exclude_ndsi_screen_fail': widgets.Checkbox(
        value=QA_CONFIG['exclude_ndsi_screen_fail'],
        description='Bit 2: Exclude NDSI Screen Fail',
        style={'description_width': 'initial'}
    ),
    'exclude_temp_height_fail': widgets.Checkbox(
        value=QA_CONFIG['exclude_temp_height_fail'],
        description='Bit 3: Exclude Temp/Height Fail',
        style={'description_width': 'initial'}
    ),
    'exclude_swir_anomaly': widgets.Checkbox(
        value=QA_CONFIG['exclude_swir_anomaly'],
        description='Bit 4: Exclude SWIR Anomaly',
        style={'description_width': 'initial'}
    ),
    'exclude_probably_cloudy': widgets.Checkbox(
        value=QA_CONFIG['exclude_probably_cloudy'],
        description='Bit 5: Exclude Probably Cloudy (v6.1)',
        style={'description_width': 'initial'}
    ),
    'exclude_probably_clear': widgets.Checkbox(
        value=QA_CONFIG['exclude_probably_clear'],
        description='Bit 6: Exclude Probably Clear (v6.1)',
        style={'description_width': 'initial'}
    ),
    'exclude_high_solar_zenith': widgets.Checkbox(
        value=QA_CONFIG['exclude_high_solar_zenith'],
        description='Bit 7: Exclude High Solar Zenith',
        style={'description_width': 'initial'}
    )
}

# Action buttons
load_data_button = widgets.Button(
    description='🔄 Load Data',
    button_style='primary',
    layout=widgets.Layout(width='150px')
)

export_params_button = widgets.Button(
    description='📤 Export Parameters',
    button_style='success',
    layout=widgets.Layout(width='200px')
)

# Output areas
stats_output = widgets.Output()
qa_output = widgets.Output()

print(\"🎛️ Interactive widgets created successfully!\")\nprint(\"   ✅ Date picker and parameter sliders\")\nprint(\"   ✅ QA level dropdown and algorithm flags\")\nprint(\"   ✅ Action buttons and output areas\")"

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════════════
# WIDGET INTERACTION FUNCTIONS
# ═══════════════════════════════════════════════════════════════════════════════════════

def load_modis_data(selected_date):
    """
    Load MODIS data for the selected date.
    """
    global current_image
    
    try:
        # Create date range (5-day window for data availability)
        start_date = ee.Date(selected_date.strftime('%Y-%m-%d'))
        end_date = start_date.advance(5, 'day')
        
        # Load MOD10A1 image
        collection = ee.ImageCollection('MODIS/061/MOD10A1') \
            .filterDate(start_date, end_date) \
            .filterBounds(glacier_geometry) \
            .select(['NDSI_Snow_Cover', 'Snow_Albedo_Daily_Tile', 
                    'NDSI_Snow_Cover_Basic_QA', 'NDSI_Snow_Cover_Algorithm_Flags_QA'])
        
        current_image = collection.first().clip(glacier_geometry)
        
        return True
        
    except Exception as e:
        print(f\"❌ Error loading MODIS data: {e}\")
        return False


def update_filtering_and_visualization():
    \"\"\"
    Update filtering and map visualization based on current widget values.
    \"\"\"
    global current_stats, current_qa_retention
    
    if current_image is None:
        with stats_output:
            clear_output()
            print(\"⚠️ Please load data first using the Load Data button.\")
        return
    
    # Get current widget values
    ndsi_threshold = ndsi_slider.value
    glacier_threshold = glacier_fraction_slider.value
    min_pixels = min_pixel_slider.value
    qa_level = qa_level_dropdown.value
    
    # Build QA configuration from checkboxes
    qa_config = {
        'basic_level': qa_level,
        **{key: checkbox.value for key, checkbox in flag_checkboxes.items()}
    }
    
    try:
        # Create quality and filtering masks
        qa_mask = create_comprehensive_quality_mask(current_image, qa_config)
        ndsi_mask = current_image.select('NDSI_Snow_Cover').gte(ndsi_threshold)
        glacier_mask = STATIC_GLACIER_FRACTION.gte(glacier_threshold / 100)
        albedo_mask = current_image.select('Snow_Albedo_Daily_Tile').lte(100)
        
        # Combined filtering mask
        combined_mask = qa_mask.And(ndsi_mask).And(glacier_mask).And(albedo_mask)
        
        # Filtered albedo
        filtered_albedo = current_image.select('Snow_Albedo_Daily_Tile') \
            .divide(100) \
            .updateMask(combined_mask) \
            .rename('filtered_albedo')
        
        # Calculate statistics
        stats = filtered_albedo.reduceRegion({
            'reducer': ee.Reducer.mean().combine(ee.Reducer.count(), '', True),
            'geometry': glacier_geometry,
            'scale': 500,
            'maxPixels': 1e9,
            'tileScale': 2
        })
        
        # Calculate QA retention rate
        total_glacier_pixels = glacier_mask.selfMask().reduceRegion({
            'reducer': ee.Reducer.count(),
            'geometry': glacier_geometry,
            'scale': 500,
            'maxPixels': 1e9
        })
        
        retained_pixels = combined_mask.selfMask().reduceRegion({
            'reducer': ee.Reducer.count(),
            'geometry': glacier_geometry,
            'scale': 500,
            'maxPixels': 1e9
        })
        
        # Get results
        stats_info = stats.getInfo()
        total_info = total_glacier_pixels.getInfo()
        retained_info = retained_pixels.getInfo()
        
        current_stats = stats_info
        current_qa_retention = {
            'total': total_info.get('constant', 0),
            'retained': retained_info.get('constant', 0)
        }
        
        # Update map layers
        update_map_layers(filtered_albedo, combined_mask)
        
        # Update statistics display
        update_statistics_display()
        
    except Exception as e:
        with stats_output:
            clear_output()
            print(f\"❌ Error in filtering update: {e}\")


def update_map_layers(filtered_albedo, mask):
    \"\"\"
    Update map layers with current filtering results.
    \"\"\"
    try:
        # Clear existing filtered layers (keep glacier mask)
        layer_names = [layer.name for layer in Map.layers[2:]]  # Skip basemap and glacier mask
        for name in layer_names:
            if 'Filtered' in name or 'QA' in name or 'NDSI' in name:
                Map.remove_layer_by_name(name)
        
        # Add filtered albedo layer
        albedo_vis = {
            'min': 0.4,
            'max': 0.9,
            'palette': ['red', 'orange', 'yellow', 'green', 'cyan', 'blue']
        }
        
        Map.addLayer(filtered_albedo, albedo_vis, f'Filtered Albedo (NDSI≥{ndsi_slider.value}, G≥{glacier_fraction_slider.value}%)')
        
        # Add QA inspection layers (hidden by default)
        Map.addLayer(current_image.select('NDSI_Snow_Cover'), 
                    {'min': 0, 'max': 100, 'palette': ['blue', 'white']}, 
                    'NDSI Snow Cover', False)
        
        Map.addLayer(current_image.select('NDSI_Snow_Cover_Basic_QA'), 
                    {'min': 0, 'max': 3, 'palette': ['green', 'yellow', 'orange', 'red']}, 
                    'Basic QA', False)
        
        Map.addLayer(current_image.select('NDSI_Snow_Cover_Algorithm_Flags_QA'), 
                    {'min': 0, 'max': 255, 'palette': ['black', 'red']}, 
                    'Algorithm Flags QA', False)
                    
    except Exception as e:
        print(f\"⚠️ Map layer update warning: {e}\")


def update_statistics_display():
    \"\"\"
    Update the statistics output display.
    \"\"\"
    with stats_output:
        clear_output()
        
        if current_stats and 'filtered_albedo_mean' in current_stats:
            mean_albedo = current_stats['filtered_albedo_mean']
            pixel_count = current_stats.get('filtered_albedo_count', 0)
            min_threshold = min_pixel_slider.value
            
            print(\"📊 Real-time Statistics (Optimized):\")
            print(\"─\" * 40)
            
            if mean_albedo is not None and pixel_count > 0:
                threshold_met = (min_threshold == 0 or pixel_count >= min_threshold)
                
                if threshold_met:
                    print(f\"✅ Mean albedo: {mean_albedo:.4f}\")
                    print(f\"📈 Qualified pixels: {pixel_count}\")
                    print(f\"🎯 Filters: NDSI≥{ndsi_slider.value} AND Glacier≥{glacier_fraction_slider.value}%\")
                    
                    if min_threshold > 0:
                        print(f\"📊 Pixel threshold (≥{min_threshold}): ✅ PASSED\")
                else:
                    print(f\"⚠️ Pixels found: {pixel_count}\")
                    print(f\"📊 Required threshold: ≥{min_threshold} pixels\")
                    print(f\"❌ Not enough qualified pixels\")
            else:
                print(f\"❌ No qualified pixels found\")
                print(f\"💡 Try reducing thresholds\")
        else:
            print(\"⏳ No statistics available\")
    
    with qa_output:
        clear_output()
        
        if current_qa_retention:
            total = current_qa_retention.get('total', 0)
            retained = current_qa_retention.get('retained', 0)
            
            if total > 0:
                retention_rate = (retained / total) * 100
                print(f\"🔍 QA Retention Rate:\")
                print(f\"📊 {retained}/{total} pixels ({retention_rate:.1f}%)\")
                print(f\"✅ QA Level: {qa_level_dropdown.value}+\")
            else:
                print(\"📊 QA retention: Calculating...\")


print(\"🎛️ Widget interaction functions defined successfully!\")
print(\"   ✅ Data loading and filtering updates\")
print(\"   ✅ Map layer management\")
print(\"   ✅ Real-time statistics display\")
print(\"   ✅ QA retention calculations\")"

## 📈 **Phase 7: Enhanced Statistical Analysis with Python**

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════════════
# ENHANCED STATISTICAL ANALYSIS - Using Python scientific libraries
# Superior to JavaScript implementation with robust statistical methods
# ═══════════════════════════════════════════════════════════════════════════════════════

def analyze_temporal_trends(data_df: pd.DataFrame, value_col: str = 'glacier_90_100pct_high_snow_mean') -> Dict:
    \"\"\"
    Comprehensive temporal trend analysis using pyMannKendall and scipy.
    
    Args:
        data_df: DataFrame with year and albedo data
        value_col: Column name containing albedo values
    
    Returns:
        Dictionary with comprehensive trend analysis results
    \"\"\"
    
    # Filter valid data
    valid_data = data_df.dropna(subset=[value_col])
    
    if len(valid_data) < 5:
        return {'error': 'Insufficient data for trend analysis (need ≥5 years)'}
    
    years = valid_data['year'].values
    values = valid_data[value_col].values
    n_years = len(years)
    
    results = {
        'dataset_info': {
            'period': f\"{years[0]}-{years[-1]}\",
            'n_years': n_years,
            'mean_albedo': float(np.mean(values)),
            'std_albedo': float(np.std(values))
        }
    }
    
    # 1. MANN-KENDALL TREND TEST (Superior to simple linear regression)
    try:
        mk_result = mk.original_test(values)
        results['mann_kendall'] = {
            'trend': mk_result.trend,
            'p_value': float(mk_result.p),
            'tau': float(mk_result.Tau),
            'z_score': float(mk_result.z),
            'slope': float(mk_result.slope) if hasattr(mk_result, 'slope') else None,
            'interpretation': 'significant' if mk_result.p < 0.05 else 'not_significant'
        }
    except Exception as e:
        results['mann_kendall'] = {'error': str(e)}
    
    # 2. SEN'S SLOPE ESTIMATOR (Robust trend magnitude)
    try:
        sens_result = mk.sens_slope(values)
        results['sens_slope'] = {
            'slope_per_year': float(sens_result.slope),
            'slope_per_decade': float(sens_result.slope * 10),
            'total_change': float(sens_result.slope * (years[-1] - years[0])),
            'intercept': float(sens_result.intercept)
        }
    except Exception as e:
        results['sens_slope'] = {'error': str(e)}
    
    # 3. AUTOCORRELATION ANALYSIS (Temporal persistence)
    if n_years >= 3:
        lag1_autocorr = np.corrcoef(values[:-1], values[1:])[0, 1]
        results['autocorrelation'] = {
            'lag1_correlation': float(lag1_autocorr),
            'persistence_level': 'HIGH' if lag1_autocorr > 0.5 else 'MODERATE' if lag1_autocorr > 0.2 else 'LOW',
            'interpretation': 'Strong year-to-year memory' if lag1_autocorr > 0.5 else \n                           'Moderate temporal dependence' if lag1_autocorr > 0.2 else \n                           'Weak temporal correlation'\n        }\n    \n    # 4. CHANGE POINT DETECTION (Enhanced method)\n    if n_years >= 10:\n        # Simple change point detection using variance\n        change_points = []\n        threshold = 2 * np.std(values)  # 2-sigma threshold\n        \n        for i in range(3, n_years - 3):\n            before_mean = np.mean(values[:i])\n            after_mean = np.mean(values[i:])\n            change_magnitude = abs(after_mean - before_mean)\n            \n            if change_magnitude > threshold:\n                change_points.append({\n                    'year': int(years[i]),\n                    'magnitude': float(change_magnitude),\n                    'direction': 'increase' if after_mean > before_mean else 'decrease'\n                })\n        \n        results['change_points'] = change_points\n    \n    # 5. ANOMALY DETECTION (Z-score method)\n    z_scores = (values - np.mean(values)) / np.std(values)\n    anomalies = []\n    \n    for i, z_score in enumerate(z_scores):\n        if abs(z_score) > 2.0:  # 2-sigma threshold\n            anomalies.append({\n                'year': int(years[i]),\n                'albedo': float(values[i]),\n                'z_score': float(z_score),\n                'type': 'HIGH' if z_score > 0 else 'LOW'\n            })\n    \n    results['anomalies'] = anomalies\n    \n    # 6. VARIABILITY ANALYSIS\n    results['variability'] = {\n        'coefficient_of_variation': float((np.std(values) / np.mean(values)) * 100),\n        'range': float(np.max(values) - np.min(values)),\n        'interquartile_range': float(np.percentile(values, 75) - np.percentile(values, 25))\n    }\n    \n    # 7. CLIMATE SIGNAL ANALYSIS (Early vs Late period)\n    if n_years >= 10:\n        split_point = n_years // 2\n        early_period = values[:split_point]\n        late_period = values[-split_point:]\n        \n        early_mean = np.mean(early_period)\n        late_mean = np.mean(late_period)\n        period_diff = late_mean - early_mean\n        relative_change = (period_diff / early_mean) * 100\n        \n        results['climate_signal'] = {\n            'early_period': {\n                'years': f\"{years[0]}-{years[split_point-1]}\",\n                'mean': float(early_mean)\n            },\n            'late_period': {\n                'years': f\"{years[-split_point]}-{years[-1]}\",\n                'mean': float(late_mean)\n            },\n            'difference': float(period_diff),\n            'relative_change_percent': float(relative_change),\n            'signal_strength': 'STRONG' if abs(relative_change) > 5 else \n                            'MODERATE' if abs(relative_change) > 2 else 'WEAK'\n        }\n    \n    return results\n\n\ndef create_comprehensive_visualization(data_df: pd.DataFrame, analysis_results: Dict, \n                                     value_col: str = 'glacier_90_100pct_high_snow_mean') -> go.Figure:\n    \"\"\"
    Create comprehensive visualization with multiple subplots using Plotly.
    
    Args:
        data_df: DataFrame with temporal data
        analysis_results: Results from analyze_temporal_trends
        value_col: Column name for albedo values
    
    Returns:
        Plotly figure with multiple subplots
    \"\"\"
    
    # Filter valid data
    valid_data = data_df.dropna(subset=[value_col])
    years = valid_data['year']
    values = valid_data[value_col]
    
    # Create subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Albedo Time Series with Trend',
            'Anomaly Detection (Z-scores)',
            'Statistical Distribution',
            'Trend Components'
        ),
        specs=[[{\"secondary_y\": False}, {\"secondary_y\": False}],
               [{\"secondary_y\": False}, {\"secondary_y\": False}]]
    )
    
    # Plot 1: Time series with trend line
    fig.add_trace(
        go.Scatter(x=years, y=values, mode='lines+markers', 
                  name='Observed Albedo', line=dict(color='blue')),
        row=1, col=1
    )
    
    # Add Sen's slope trend line if available
    if 'sens_slope' in analysis_results and 'slope_per_year' in analysis_results['sens_slope']:
        slope = analysis_results['sens_slope']['slope_per_year']
        intercept = analysis_results['sens_slope']['intercept']
        trend_line = slope * (years - years.iloc[0]) + values.iloc[0]
        
        fig.add_trace(
            go.Scatter(x=years, y=trend_line, mode='lines', 
                      name=\"Sen's Slope Trend\", line=dict(color='red', dash='dash')),
            row=1, col=1
        )
    
    # Plot 2: Anomaly detection
    z_scores = (values - values.mean()) / values.std()
    colors = ['red' if abs(z) > 2 else 'blue' for z in z_scores]
    
    fig.add_trace(
        go.Scatter(x=years, y=z_scores, mode='markers', 
                  marker=dict(color=colors, size=8),
                  name='Z-scores'),
        row=1, col=2
    )
    
    # Add threshold lines
    fig.add_hline(y=2, line_dash=\"dash\", line_color=\"red\", row=1, col=2)
    fig.add_hline(y=-2, line_dash=\"dash\", line_color=\"red\", row=1, col=2)
    
    # Plot 3: Distribution
    fig.add_trace(
        go.Histogram(x=values, nbinsx=10, name='Distribution', 
                    marker_color='lightblue', opacity=0.7),
        row=2, col=1
    )
    
    # Plot 4: Trend components (if Mann-Kendall available)
    if 'mann_kendall' in analysis_results and 'tau' in analysis_results['mann_kendall']:
        components = ['Tau', 'P-value', 'Z-score']
        mk_results = analysis_results['mann_kendall']
        comp_values = [mk_results.get('tau', 0), mk_results.get('p_value', 0), mk_results.get('z_score', 0)]
        
        fig.add_trace(
            go.Bar(x=components, y=comp_values, name='Mann-Kendall Stats',
                  marker_color=['green', 'orange', 'purple']),
            row=2, col=2
        )
    
    # Update layout
    fig.update_layout(
        title_text=\"Comprehensive Albedo Trend Analysis\",
        height=800,
        showlegend=True
    )
    
    # Update axes labels
    fig.update_xaxes(title_text=\"Year\", row=1, col=1)
    fig.update_yaxes(title_text=\"Albedo\", row=1, col=1)
    fig.update_xaxes(title_text=\"Year\", row=1, col=2)
    fig.update_yaxes(title_text=\"Z-score\", row=1, col=2)
    fig.update_xaxes(title_text=\"Albedo\", row=2, col=1)
    fig.update_yaxes(title_text=\"Frequency\", row=2, col=1)
    fig.update_xaxes(title_text=\"Component\", row=2, col=2)
    fig.update_yaxes(title_text=\"Value\", row=2, col=2)
    
    return fig


print(\"📈 Enhanced statistical analysis functions defined successfully!\")
print(\"   🔬 Mann-Kendall trend test (non-parametric)\")
print(\"   📐 Sen's slope estimator (robust trends)\")
print(\"   🔄 Autocorrelation analysis (temporal persistence)\")
print(\"   🎯 Change point detection (structural breaks)\")
print(\"   ⚠️ Anomaly detection (z-score method)\")
print(\"   📊 Comprehensive visualization (Plotly subplots)\")
print(\"   🌡️ Climate signal analysis (early vs late period)\")"